# 🏔️ The Live Folding Landscape
### Visualizing the Energy Funnel Theory of Protein Folding

In this tutorial, we explore one of the most profound concepts in structural biology: the **Energy Funnel**. Proteins do not fold by random searching (which would take longer than the age of the universe, known as **Levinthal's Paradox**). Instead, they follow a rugged energy landscape that guides them toward their most stable, "native" state.

Specifically, we will:
1. Generate a Ramachandran-style **Energy Surface** for a peptide.
2. Run a **Simulated Annealing** trajectory using the `synth-pdb` physics engine.
3. Map the folding trajectory onto the 3D energy surface.

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
import os, sys, numpy as np
import py3Dmol
import plotly.graph_objects as go
import plotly.io as pio

# --- UNIVERSAL SETUP ---
# Letting Plotly auto-detect the best renderer for the environment
sys.path.append(os.path.abspath('../../'))
from synth_pdb import PeptideGenerator, EnergyMinimizer, PDBValidator, PeptideResult
import biotite.structure as struc

# Init Plotly variables to prevent NameError
X, Y, Z = np.array([]), np.array([]), np.array([])
traj_phi, traj_psi, energies = np.array([]), np.array([]), []
trajectory_structs = []

print("[✅] Environment Ready.")

## 1. Creating the Energy Landscape
We'll take a small peptide and systematically rotate the Dihedral Angles (Phi and Psi) of a central residue. For each combination, we'll calculate the **Potential Energy** to build a 3D landscape.

In [ ]:
def get_energy_for_angles(phi, psi, sequence="ALA-ALA-ALA"):
    """Calculates energy for a given phi/psi of the central residue."""
    gen = PeptideGenerator(sequence)
    # Ideal alpha helix base
    phis = [-57.0, phi, -57.0]
    psis = [-47.0, psi, -47.0]
    
    try:
        # Regenerate with specific central angles
        res = gen.generate(phi_list=phis, psi_list=psis)
        energy = minimizer.calculate_energy(res)
        return energy if energy is not None else 10000.0
    except:
        return 10000.0


In [ ]:
minimizer = EnergyMinimizer()

# Generate Grid
res_grid = 15
phis = np.linspace(-180, 180, res_grid)
psis = np.linspace(-180, 180, res_grid)
X, Y = np.meshgrid(phis, psis)
Z = np.zeros_like(X)

print("Generating Energy Landscape... (approx 30s)")
for i in range(res_grid):
    for j in range(res_grid):
        val = get_energy_for_angles(X[i,j], Y[i,j])
        Z[i,j] = min(val, 5000)
            
print("[✅] Energy Landscape Ready.")

## 2. Running the Folding Trajectory
Now we run a series of **Energy Minimization** steps. We start at a "Random" state and allow the protein to relax. We'll capture snapshots along the way.

In [ ]:
sequence = "ALA-ALA-ALA-ALA-ALA"
gen = PeptideGenerator(sequence)
res = gen.generate(conformation="random") 

trajectory_structs = []
energies = []
phi_psi_history = []

minimizer = EnergyMinimizer()
validator = PDBValidator(res.pdb)

print("Folding Peptide...")
import tempfile

for step in range(15):
    # Standardize on file-based minimize for multi-step refinement
    with tempfile.NamedTemporaryFile(suffix='.pdb', mode='w', delete=False) as f_in:
        f_in.write(res.pdb); f_in.close()
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False) as f_out:
            f_out.close()
            minimizer.minimize(f_in.name, f_out.name, max_iterations=20)
            with open(f_out.name, 'r') as r: updated_pdb = r.read()
            res = PeptideResult(updated_pdb) # Re-wrap
            os.unlink(f_in.name); os.unlink(f_out.name)
    
    trajectory_structs.append(res.structure.copy())
    energies.append(minimizer.calculate_energy(res))
    print(f"Refining Progress: Step {step+1}/15 | Energy: {energies[-1]:.2f} kJ/mol")
    
    # Record Phi/Psi of a central residue
    angles = validator.calculate_dihedrals(res)
    phi_psi_history.append([angles['phi'][2], angles['psi'][2]])

traj_phi = np.array([p[0] for p in phi_psi_history])
traj_psi = np.array([p[1] for p in phi_psi_history])
print(f"Folding Complete. Final Energy: {energies[-1]:.2f} kJ/mol")

## 3. The Interactive 3D Funnel Plot
This plot combines the energy surface (static) with the trajectory (dynamic) of the folding protein.

In [ ]:
if len(X) == 0 or len(traj_phi) == 0:
    print("Waiting for landscape generation (Section 1) and folding simulation (Section 2) to complete...")
else:
    fig = go.Figure(data=[
        go.Surface(z=Z, x=X, y=Y, colorscale='Blues', opacity=0.7, name='Energy Surface'),
        go.Scatter3d(x=traj_phi, y=traj_psi, z=energies, 
                     mode='markers+lines', 
                     marker=dict(size=6, color='orange', symbol='diamond'),
                     line=dict(color='orange', width=4), 
                     name='Folding Path')
    ])

    fig.update_layout(title='Protein Folding Energy Landscape', 
                      scene=dict(xaxis_title='Phi (Degrees)', yaxis_title='Psi (Degrees)', zaxis_title='Potential Energy (kJ/mol)'),
                      width=900, height=800, template="plotly_dark")
    fig.show()

## 4. Visualizing the Molecule
Finally, browse through the snapshots to see the backbone actually condensing into its stable form.

In [ ]:
import py3Dmol

viewer_output = widgets.Output()
info_label = widgets.HTML("<b>Select a snapshot to begin</b>")

def display_molecule(index):
    """Render using the py3Dmol package directly."""
    info_label.value = f"""
    <div style='background: #333; color: white; padding: 12px; border-radius: 8px; font-family: sans-serif;'>
    <b>Snapshot {index}</b> | Energy: <span style='color: #00ffcc;'>{energies[index]:.1f} kJ/mol</span>
    </div>
    """
    
    with viewer_output:
        clear_output(wait=True)
        # Use biotite to get PDB string
        from biotite.structure.io.pdb import PDBFile
        import io
        f = PDBFile()
        f.set_structure(trajectory_structs[index])
        sink = io.StringIO()
        f.write(sink)
        pdb_str = sink.getvalue()
        
        view = py3Dmol.view(width=500, height=400)
        view.addModel(pdb_str, 'pdb')
        view.setStyle({'stick': {'color': 'spectrum'}, 'sphere': {'scale': 0.3}})
        view.zoomTo()
        display(view.show())

def on_slider_change(change):
    display_molecule(change['new'])

slider = widgets.IntSlider(
    value=0, min=0, max=max(0, len(trajectory_structs)-1), step=1, 
    description='Snapshot:', continuous_update=False,
    layout=widgets.Layout(width='500px')
)
slider.observe(on_slider_change, names='value')

display(widgets.VBox([info_label, slider, viewer_output]))
if len(trajectory_structs) > 0: display_molecule(0)
